In [1]:
#GOALS
#Create environment (perhaps at a pixel level) for simple drones to fly around in
#Create leader and follower drones
#Create destination
#Reward drones for moving towards destination (like in Hidden Identity paper)
#Create obstacle
#Punish drones for moving too close to obstacle?
#Find some way to reward drones for flying in formation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pygame
import random


pygame 2.1.2 (SDL 2.0.16, Python 3.8.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from pygame.locals import (
    RLEACCEL,
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)

In [3]:
pygame.init()

screen = pygame.display.set_mode([500, 500])

running = True
while running:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill((255,255,255))

    pygame.draw.circle(screen, (0,0,255), (250,250), 75)

    pygame.display.flip()

pygame.quit()

In [4]:
pygame.init()

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super(Player, self).__init__()
        self.surf = pygame.Surface((75, 25))
        self.surf.fill((255, 255, 255))
        self.rect = self.surf.get_rect()

    def update(self, pressed_keys):
        if pressed_keys[K_UP]:
            self.rect.move_ip(0,-5)
        if pressed_keys[K_DOWN]:
            self.rect.move_ip(0,5)
        if pressed_keys[K_LEFT]:
            self.rect.move_ip(-5,0)
        if pressed_keys[K_RIGHT]:
            self.rect.move_ip(5,0)

        if self.rect.left < 0:
            self.rect.left = 0
        if self.rect.right > SCREEN_WIDTH:
            self.rect.right = SCREEN_WIDTH
        if self.rect.top <= 0:
            self.rect.top = 0
        if self.rect.bottom >= SCREEN_HEIGHT:
            self.rect.bottom = SCREEN_HEIGHT

class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super(Enemy, self).__init__()
        self.surf = pygame.Surface((20,10))
        self.surf.fill((255,0,0))
        self.rect = self.surf.get_rect(
            center=(
                random.randint(SCREEN_WIDTH + 20, SCREEN_WIDTH + 100),
                random.randint(0, SCREEN_HEIGHT),
            )
        )
        self.speed = random.randint(5,20)

    def update(self):
        self.rect.move_ip(-self.speed, 0)
        if self.rect.right < 0:
            self.kill()

clock = pygame.time.Clock()

SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

ADDENEMY = pygame.USEREVENT + 1
pygame.time.set_timer(ADDENEMY, 250)

player = Player()

enemies = pygame.sprite.Group()
all_sprites = pygame.sprite.Group()
all_sprites.add(player)

running = True

while running:
    for event in pygame.event.get():
        if event.type == KEYDOWN:
            if event.key == K_ESCAPE:
                running = False

        elif event.type == QUIT:
            running = False

        elif event.type == ADDENEMY:
            new_enemy = Enemy()
            enemies.add(new_enemy)
            all_sprites.add(new_enemy)

    pressed_keys = pygame.key.get_pressed()

    player.update(pressed_keys)

    enemies.update()

    screen.fill((0,0,0))

    for entity in all_sprites:
        screen.blit(entity.surf, entity.rect)

    if pygame.sprite.spritecollideany(player, enemies):
        player.kill()
        running = False

    pygame.display.flip()

    clock.tick(60)

pygame.quit()

In [68]:
#pygame.init()

class Drone(pygame.sprite.Sprite):
    def __init__(self, xi, yi, zi, mode):
        self.x = xi
        self.y = yi
        self.z = zi
        self.mode = mode
        super(Drone, self).__init__()
        self.surf = pygame.Surface((10, 10))
        self.surf.fill((0, 191, 255))
        self.rect = self.surf.get_rect(
            center=(xi,yi,)
        )

        self.direction = np.pi / 2
        #self.direction = random.randint(0,43) / 7
        self.force = 10

        self.l = self.rect.left
        self.top = self.rect.top

    def update(self):

        if self.mode==0:
            self.direction=0
            self.force=0
        elif self.mode==1:
            self.direction = random.randint(0,43) / 7
            self.force = 10
        elif self.mode==2:
            self.direction = 0
            self.force = 10

        if self.force > 0:
            dx = round(self.force * np.cos(self.direction))
            dy = -1*round(self.force * np.sin(self.direction))
            self.rect.move_ip(dx,dy)

        if self.rect.left < 0:
            self.rect.left = 0
        if self.rect.right > SCREEN_WIDTH:
            self.rect.right = SCREEN_WIDTH
        if self.rect.top <= 0:
            self.rect.top = 0
        if self.rect.bottom >= SCREEN_HEIGHT:
            self.rect.bottom = SCREEN_HEIGHT

        self.l = self.rect.left
        self.t = self.rect.top

    def observe(self):
        tophat = self.rect.top + 10
        scanup = Scanner(self.rect.left, tophat, 10, 1, "u")
        return "observed"

class Goal(pygame.sprite.Sprite):
    def __init__(self, x, y, z):
        super(Goal, self).__init__()
        self.surf = pygame.Surface((20, 20))
        self.surf.fill((57,255,20))
        self.rect = self.surf.get_rect(
            center=(x,y,)
        )

class Obstacle(pygame.sprite.Sprite):
    def __init__(self,x,y,z,xdim,ydim,zdim):
        super(Obstacle, self).__init__()
        self.surf = pygame.Surface((xdim,ydim))
        self.surf.fill((255,0,0))
        self.rect = self.surf.get_rect(
            center=(x,y,)
        )

class Scanner(pygame.sprite.Sprite):
    def __init__(self,left,top,xdim,ydim,dir):
        super(Scanner, self).__init__()
        self.surf = pygame.Surface((xdim,ydim))
        self.surf.fill((255,255,255))
        self.rect = self.surf.get_rect()
        self.rect.left = left
        self.rect.top = top
        self.dir = dir

    def update(self):
        if self.dir == "r":
            self.rect.move_ip(1,0)
        elif self.dir == "l":
            self.rect.move_ip(-1,0)
        elif self.dir == "u":
            self.rect.move_ip(0,-1)
        elif self.dir == "d":
            self.rect.move_ip(0,1)

        

            

#clock = pygame.time.Clock()

#SCREEN_WIDTH = 800
#SCREEN_HEIGHT = 600

#screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

#d1 = Drone(100,300,0)
#d2 = Drone(50,200,0)
#d3 = Drone(75,250,0)
#d4 = Drone(60,300,0)
#d5 = Drone(90,400,0)
#d6 = Drone(100,500,0)

#drones = pygame.sprite.Group()
#drones.add(d1,d2,d3,d4,d5,d6)

#running = True

#counter = 0

#while running:
#    for event in pygame.event.get():
#        if event.type == KEYDOWN:
#            if event.key == K_ESCAPE:
#                running = False
#
#        elif event.type == QUIT:
#            running = False
#
#    drones.update()
#
#    screen.fill((0,0,0))
#
#    for entity in drones:
#        screen.blit(entity.surf, entity.rect)
#        for entity2 in drones:
#            if entity != entity2 and pygame.sprite.collide_rect(entity, entity2):
#                entity.kill()
#                entity2.kill()
#
#    pygame.display.flip()
#
#    counter += 1
#    if counter >= 500:
#        running = False
#
#    clock.tick(60)



#pygame.quit()

In [73]:
def onesim(maxlen):
    pygame.init()

    clock = pygame.time.Clock()

    #Sets the height and width of the environment
    SCREEN_WIDTH = 800
    SCREEN_HEIGHT = 600

    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

    # output array is num_frames*num_drones*num_coords
    posarr = np.zeros((maxlen,6,3))

    allsprites = pygame.sprite.Group()

    # 6 drones are currently hardcoded in with starting positions
    d1 = Drone(100,300,0,1)
    d2 = Drone(50,200,0,1)
    d3 = Drone(75,250,0,1)
    d4 = Drone(60,300,0,2)
    d5 = Drone(90,400,0,2)
    d6 = Drone(100,500,0,2)

    drones = pygame.sprite.Group()
    drones.add(d1,d2,d3,d4,d5,d6)
    allsprites.add(d1,d2,d3,d4,d5,d6)

    dronelist = [d1,d2,d3,d4,d5,d6]

    # A green 'goal' is randomly placed somehwere on the right side of the environment
    ygoal = np.random.randint(150,450)
    endgoal = Goal(750,ygoal,0)

    goals = pygame.sprite.Group()
    goals.add(endgoal)
    allsprites.add(endgoal)

    # Two obstacles randomly placed in the middle of the environment
    yo1 = np.random.randint(150,450)
    o1 = Obstacle(350,yo1,0,20,300,0)

    yo2 = np.random.randint(150,450)
    o2 = Obstacle(550,yo2,0,20,300,0)

    obstacles = pygame.sprite.Group()
    obstacles.add(o1,o2)
    allsprites.add(o1,o2)

    # If running is ever false, the run terminates
    running = True
    counter = 0

    scans = pygame.sprite.Group()

    while running:
        # Can force exit the simulation using ESC or by closing the window
        for event in pygame.event.get():
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    running = False

            elif event.type == QUIT:
                running = False


        # Drones update, according to the update function defined in Drone class
        drones.update()


        screen.fill((0,0,0))

        for entity in allsprites:
            screen.blit(entity.surf, entity.rect)
            
        for entity in drones:
            # Drone is killed if it collides with an obstacle
            if pygame.sprite.spritecollideany(entity, obstacles):
                entity.kill()

            # Drone is killed if it collides with the goal (success!)
            if pygame.sprite.spritecollideany(entity,goals):
                entity.kill()

            # Both drones are killed if they collide with each other
            for entity2 in drones:
                if entity != entity2 and pygame.sprite.collide_rect(entity, entity2):
                    entity.kill()
                    entity2.kill()

        for entity in drones:
            l=entity.rect.left
            lx=l-1
            r=entity.rect.right
            rx=r+1
            t=entity.rect.top
            tx=t-1
            b=entity.rect.bottom
            bx=b+1

            results = [[0,0],[0,0],[0,0],[0,0]]

            rscan = Scanner(r,t,1,10,"r")
            scans.add(rscan)

            for i in range(100):
                rscan.update()
                print(rscan.rect.left)

            for entity in scans:
                entity.kill()
            

        pygame.display.flip()

        dead = 0

        # Store results in posarr
        for i in range(len(dronelist)):
            entity = dronelist[i]
            if entity.alive():
                posarr[counter,i] = [entity.rect.left,entity.rect.top,0]
            else:
                dead += 1
                posarr[counter,i] = [-1,-1,-1]

        # Run ends early if all drones are dead
        if dead == 6:
            running = False

        # Run ends if max length has been reached
        counter += 1
        if counter >= maxlen:
            running = False

        # I'm not sure if this is helping the code run more smoothly or not
        clock.tick(60)

    pygame.quit()
    
    return posarr

In [74]:
x=onesim(100)

96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138


In [25]:
print(x[99])

[[152.   1.   0.]
 [ 35.   1.   0.]
 [184.   1.   0.]
 [ -1.  -1.  -1.]
 [ -1.  -1.  -1.]
 [ -1.  -1.  -1.]]


In [42]:
def multisim(numsims,maxlen):
    multisimlist = []
    for i in range(numsims):
        x=onesim(maxlen)
        multisimlist.append(x)
    multisimlist=np.array(multisimlist)

    return multisimlist

In [44]:
z=multisim(3,100)

In [40]:
np.shape(z)

(3, 100, 6, 3)